![CH8-ADS.png](.\Media\CH8-ADS.png)
# <span style="color:#cc5500;">Deploy Azure SQL VM from PowerShell </span>

In this Notebook, we will eplore you you can deploy and Azure SQL VM from PowerShell.

Why is this covered?  Most people when the deploy Azure Services for the first time, use the Azure Portal just like we did in the previous notebook.  The Portal is great for learning how SQL Server is installed to an Azure VM with the many menu-driven options and drop-down selections.  Using the Portal is likely the best option if you just need to get something deployed and won't have to do it more than once or twice.  However, when using the Portal, it is very easy to deploy your SQL Servers 'differently'.  For example, maybe the first time you deployed SQL, you chose a SQL Server 2016 Enterprise Edition with HDD disks on Windows Server 2016.  Then, you deploy a 2nd SQL Server, but you didn't realize that you chose a different version of SQL and you used Premium SSD disks.  Generally in an enterprise, you want all things to be consistent so that you can have good control of performance charasteristics.

This is the beauty of IaaC (Infrastructure as Code).  You create repeatable scripts, so that you have a consistent deployments over and over again.

Unlike the previous chapters in this notebook, you will not be using the SQL kernel in the code blocks.  Rather, in the cell blocks that follow make sure you have the PowerShell Kernel selected.

Azure Data Studio, by default, is pre-set to connect to our Commercial Azure Data Centers.  However, there are Sovereign Cloud data centers as well like Azure US Government.  I have seen very few youtube videos and on-line articles that discuss this.  As such, I will demonstrate the nuances of getting Azure Data Studio connected to a US Government Data Center where I happen to have a subscription.  If you only have a subscription in Azure Commercial, this script will work, you will just need to make the appropriate adustments to the code below when I prompt you to.

First go to the Settings Gear on the lower Left and select Settings \> Extensions \> Azure Account Configuration and make sure you have the desired "Soverign" Cloud checked if you have a subscription there.  If not, you can just use the default option.

![CH8-AzureAccount.jpg](.\Media\CH8-AzureAccount.jpg)


In [1]:
# Make Initial Connection.  Azure Data Studio connects to Azure Commercial data centers.
# If connecting to Azure Gov, it must be specified in the -EnviornmentName, otherwise remove that and just use Connect-AzAccount
# When you run this command, have your account ready and your password ready.  It will pop open a window for you to enter your credentials

Connect-AzAccount -EnvironmentName AzureUSGovernment 

selected for further use. To select another subscription, use Set-AzContext.


Account                                    SubscriptionName      TenantId                             Environment      
-------                                    ----------------      --------                             -----------      
kewarren@mcsinternaltrials.onmicrosoft.com Kevin Warren-MCSTrial 8a09f2d7-8415-4296-92b2-80bb4666c5fc AzureUSGovernment




In [2]:
# If you forget to set the scope (or context) of the Azure PowerSell Az commands to the correct Azure Subscription, then you may end up provisioning or deleting resources in the wrong Azure Subscription
# From the results from the above command, enter the Subscription name below in this block.

Set-AzContext -SubscriptionName "US Gov FedCon"

. {


>> # If you forget to set the scope (or context) of the Azure PowerSell Az commands to the correct Azure Subscription, then you may end up provisioning or deleting resources in the wrong Azure Subscription
>> # From the results from the above command, enter the Subscription name below in this block.
>> 
>> Set-AzContext -SubscriptionName "US Gov FedCon"
>> }
>> 



Name                                     Account             SubscriptionName    Environment         TenantId          
----                                     -------             ----------------    -----------         --------          
US Gov FedCon (714fe24d-b28a-49e3-9ad... kewarren@mcsinte... US Gov FedCon       AzureUSGovernment   8a09f2d7-8415-4...


In [3]:
# Since I have multiple subscriptions, I need a list so that I can pick the one I want to use
Get-AzSubscription

Name                  Id                                   TenantId                             State  
----                  --                                   --------                             -----  
Kevin Warren-MCSTrial 7ee690f2-39eb-4244-baf2-697428b4650f 8a09f2d7-8415-4296-92b2-80bb4666c5fc Enabled
US Gov FedCon         714fe24d-b28a-49e3-9adf-a63a6ec1878d 8a09f2d7-8415-4296-92b2-80bb4666c5fc Enabled




In [4]:
# Get a list of Azure Locations so you know which location to add to your globar variable

Get-AzLocation



Location    : usgovvirginia
DisplayName : USGov Virginia
Providers   : {Microsoft.Network, Microsoft.Security, Microsoft.Compute, Microsoft.CognitiveServices...}

Location    : usgoviowa
DisplayName : USGov Iowa
Providers   : {Microsoft.Network, Microsoft.Security, Microsoft.Compute, Microsoft.Storage...}

Location    : usdodeast
DisplayName : USDoD East
Providers   : {Microsoft.Network, Microsoft.Compute, Microsoft.Media, Microsoft.Storage...}

Location    : usdodcentral
DisplayName : USDoD Central
Providers   : {Microsoft.Network, Microsoft.Compute, Microsoft.Media, Microsoft.EventGrid...}

Location    : usgovtexas
DisplayName : USGov Texas
Providers   : {Microsoft.Network, Microsoft.Security, Microsoft.Compute, Microsoft.Media...}

Location    : usgovarizona
DisplayName : USGov Arizona
Providers   : {Microsoft.Network, Microsoft.Security, Microsoft.Compute, Microsoft.CognitiveServices...}



In [6]:
## Set Global variables

$Location = "usgovvirginia"
$ResourceGroupName = "rgazsqlvm"


In [7]:
## Storage
$StorageName = $ResourceGroupName + "storage"
$StorageSku = "Premium_LRS"

In [10]:
## Network Properties 

$InterfaceName = $ResourceGroupName + "ServerInterface"
$NsgName = $ResourceGroupName + "nsg"
$VNetName = $ResourceGroupName + "VNet"
$SubnetName = "Default"
$VNetAddressPrefix = "10.0.0.0/16"
$VNetSubnetAddressPrefix = "10.0.0.0/24"
$TCPIPAllocationMethod = "Dynamic"
$DomainName = $ResourceGroupName

In [11]:
##Compute

$VMName = $ResourceGroupName + "VM"
$ComputerName = $ResourceGroupName + "Server"
$VMSize = "Standard_DS1_V2"
$OSDiskName = $VMName + "OSDisk"

In [12]:
##Image

$PublisherName = "MicrosoftSQLServer"
$OfferName = "SQL2017-WS2016"
$Sku = "SQLDEV"
$Version = "latest"


In [13]:
# Resource Group
New-AzResourceGroup -Name $ResourceGroupName -Location $Location



ResourceGroupName : rgazsqlvm
Location          : usgovvirginia
ProvisioningState : Succeeded
Tags              : 
ResourceId        : /subscriptions/714fe24d-b28a-49e3-9adf-a63a6ec1878d/resourceGroups/rgazsqlvm



In [14]:
# Storage  This may take a few minutes to complete
$StorageAccount = New-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name $StorageName -SkuName $StorageSku -Kind "Storage" -Location $Location

In [15]:
# Network

$SubnetConfig = New-AzVirtualNetworkSubnetConfig -Name $SubnetName -AddressPrefix $VNetSubnetAddressPrefix
$VNet = New-AzVirtualNetwork -Name $VNetName -ResourceGroupName $ResourceGroupName -Location $Location -AddressPrefix $VNetAddressPrefix -Subnet $SubnetConfig
$PublicIp = New-AzPublicIpAddress -Name $InterfaceName -ResourceGroupName $ResourceGroupName -Location $Location -AllocationMethod $TCPIPAllocationMethod -DomainNameLabel $DomainName
$NsgRuleRDP = New-AzNetworkSecurityRuleConfig -Name "RDPRule" -Protocol Tcp -Direction Inbound -Priority 1000 -SourceAddressPrefix * -SourcePortRange * -DestinationAddressPrefix * -DestinationPortRange 3389 -Access Allow
$NsgRuleSQL = New-AzNetworkSecurityRuleConfig -Name "MSSQLRule"  -Protocol Tcp -Direction Inbound -Priority 1001 -SourceAddressPrefix * -SourcePortRange * -DestinationAddressPrefix * -DestinationPortRange 1433 -Access Allow
$Nsg = New-AzNetworkSecurityGroup -ResourceGroupName $ResourceGroupName -Location $Location -Name $NsgName -SecurityRules $NsgRuleRDP,$NsgRuleSQL
$Interface = New-AzNetworkInterface -Name $InterfaceName -ResourceGroupName $ResourceGroupName -Location $Location -SubnetId $VNet.Subnets[0].Id -PublicIpAddressId $PublicIp.Id -NetworkSecurityGroupId $Nsg.Id

Update Property Name
Cmdlet invocation changes :
    Old Way : -ResourceId
    New Way : -NatGatewayId
Update Property Name
Cmdlet invocation changes :
    Old Way : -InputObject
    New Way : -NatGateway
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other 
information on breaking changes in Azure PowerShell.


Default behaviour of Zone will be changed
Cmdlet invocation changes :
    Old Way : Sku = Standard means the Standard Public IP is zone-redundant.
    New Way : Sku = Standard and Zone = {} means the Standard Public IP has no zones. If you want to create a 
zone-redundant Public IP address, please specify all the zones in the region. For example, Zone = ['1', '2', '3'].
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other 
information on breaking changes in Azure PowerShell.


In [16]:
# Compute.  This will prompt you to enter a username and password

$VirtualMachine = New-AzVMConfig -VMName $VMName -VMSize $VMSize
$Credential = Get-Credential -Message "Type the name and password of the local administrator account."
$VirtualMachine = Set-AzVMOperatingSystem -VM $VirtualMachine -Windows -ComputerName $ComputerName -Credential $Credential -ProvisionVMAgent -EnableAutoUpdate #-TimeZone = $TimeZone
$VirtualMachine = Add-AzVMNetworkInterface -VM $VirtualMachine -Id $Interface.Id
$OSDiskUri = $StorageAccount.PrimaryEndpoints.Blob.ToString() + "vhds/" + $OSDiskName + ".vhd"
$VirtualMachine = Set-AzVMOSDisk -VM $VirtualMachine -Name $OSDiskName -VhdUri $OSDiskUri -Caching ReadOnly -CreateOption FromImage

In [17]:
# Image
$VirtualMachine = Set-AzVMSourceImage -VM $VirtualMachine -PublisherName $PublisherName -Offer $OfferName -Skus $Sku -Version $Version

In [18]:
# Create the VM in Azure  Thows errors, comment on this.  Takes a few minutes to run
New-AzVM -ResourceGroupName $ResourceGroupName -Location $Location -VM $VirtualMachine

diagnostics.  : Microsoft.Rest.ValidationException: 'Kind' cannot be null.
   at Microsoft.Azure.PowerShell.Cmdlets.Compute.Helpers.Storage.Models.StorageAccountCreateParameters.Validate()
   at 
Microsoft.Azure.PowerShell.Cmdlets.Compute.Helpers.Storage.StorageAccountsOperations.<BeginCreateWithHttpMessagesAsync>
d__11.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at System.Runtime.ExceptionServices.ExceptionDispatchInfo.Throw()
   at System.Runtime.CompilerServices.TaskAwaiter.HandleNonSuccessAndDebuggerNotification(Task task)
   at 
Microsoft.Azure.PowerShell.Cmdlets.Compute.Helpers.Storage.StorageAccountsOperations.<CreateWithHttpMessagesAsync>d__6.
MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at System.Runtime.ExceptionServices.ExceptionDispatchInfo.Throw()
   at System.Runtime.CompilerServices.TaskAwaiter.HandleNonSuccessAndDebuggerNotification(Task task)
   at 
Microsoft.Azure.PowerShell.C

RequestId IsSuccessStatusCode StatusCode ReasonPhrase
--------- ------------------- ---------- ------------
                         True         OK OK          




In [19]:
# Add the SQL IaaS Extension, and choose the license type.  Takes a few minutes to run

New-AzSqlVM -ResourceGroupName $ResourceGroupName -Name $VMName -Location $Location -LicenseType PAYG

. {
>> # Add the SQL IaaS Extension, and choose the license type
>> 

New-AzSqlVM -ResourceGroupName $ResourceGroupName -Name $VMName -Location $Location -LicenseType PAYG
>> }
>> 


Name        ResourceGroupName LicenseType Sku       Offer          SqlManagementType
----        ----------------- ----------- ---       -----          -----------------
rgazsqlvmVM rgazsqlvm         PAYG        Developer SQL2017-WS2016 LightWeight      


